<img src="../Pierian-Data-Logo.PNG">
<br>
<strong><center>Copyright 2019. Created by Jose Marcial Portilla.</center></strong>

# What is CUDA?

Most people confuse CUDA for a language or maybe an API. It is not.

It’s more than that. CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple and elegant. The developer still programs in the familiar C, C++, Fortran, or an ever expanding list of supported languages, and incorporates extensions of these languages in the form of a few basic keywords.

These keywords let the developer express massive amounts of parallelism and direct the compiler to the portion of the application that maps to the GPU.

# How do I install PyTorch for GPU?

Refer to video, its dependent on whether you have an NVIDIA GPU card or not.

# How do I know if I have CUDA available?

In [1]:
import torch
torch.cuda.is_available()
# True

True

# Using GPU and CUDA

We've provided 2 versions of our yml file, a GPU version and a CPU version. To use GPU, you need to either manually create a virtual environment, please watch the video related to this lecture, as not every computer can run GPU, you need CUDA and an NVIDIA GPU.

In [2]:
## Get Id of default device
torch.cuda.current_device()

0

In [3]:
# 0
torch.cuda.get_device_name(0) # Get name device with ID '0'

'NVIDIA GeForce RTX 3050 Ti Laptop GPU'

In [4]:
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
torch.cuda.memory_allocated()

0

In [6]:
# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
torch.cuda.memory_reserved()

0

# Using CUDA instead of CPU

In [7]:
# CPU
a = torch.FloatTensor([1.,2.])

In [8]:
a

tensor([1., 2.])

In [9]:
a.device

device(type='cpu')

In [10]:
# GPU
a = torch.FloatTensor([1., 2.]).cuda()

In [11]:
a.device

device(type='cuda', index=0)

In [12]:
torch.cuda.memory_allocated()

512

## Sending Models to GPU

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [15]:
torch.manual_seed(32)
model = Model()

In [16]:
# From the discussions here: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda

False

In [17]:
gpumodel = model.cuda()

In [18]:
next(gpumodel.parameters()).is_cuda

True

In [19]:
df = pd.read_csv('../Data/iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

## Convert Tensors to .cuda() tensors

In [20]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [21]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [23]:
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15071154
epoch: 11  loss: 0.93773144
epoch: 21  loss: 0.77982575
epoch: 31  loss: 0.60994011
epoch: 41  loss: 0.40079930
epoch: 51  loss: 0.25436315
epoch: 61  loss: 0.15053052
epoch: 71  loss: 0.10086944
epoch: 81  loss: 0.08128313
epoch: 91  loss: 0.07231426
TOTAL TRAINING TIME: 0.6543569564819336
